# Численное интегрирование



## Метод прямоугольников

Сумма Римана: $$\int_a^b f(x) dx \approx \sum_{i=1}^n f(x^*_i) \Delta x_i,\ \Delta x_i = x_i - x_{i-1}$$


### Метод левых прямоугольников (левая сумма)

В качестве $x^*_i$ выбираются левые границы отрезков: $x^*_i = x_{i-1}$.

In [1]:
import numpy as np

def left_riemann_sum(f, a, b, n):
  def x(i): return a + i*((b - a) / n)

  return sum(f(x(i-1)) * (x(i) - x(i-1)) for i in range(0, n + 1))

### Метод правых прямоугольников (правая сумма)

В качестве $x^*_i$ выбираются правые границы отрезков: $x^*_i = x_{i}$.

In [2]:
def right_riemann_sum(f, a, b, n):
  def x(i): return a + i*((b - a) / n)

  return sum(f(x(i)) * (x(i) - x(i-1)) for i in range(0, n + 1))

## Точность расчетов

In [3]:
def iterate_upto_precision(method, f, a, b, precision=10**-6):
  n = 4
  res_n, res2n = 0, 0
  while True:
    res_n = method(f, a, b, n)
    res_2n = method(f, a, b, 2*n)
    if abs(res_n - res_2n) > precision:
      n *= 2
    else:
      return n, res_2n

## Ввод формул

In [4]:
from sympy.parsing.sympy_parser import \
  parse_expr, standard_transformations, implicit_multiplication_application, implicit_application, convert_xor
from sympy.utilities.autowrap import autowrap

def parse_function(raw_expr):
  transformations = (standard_transformations +
    (implicit_multiplication_application, implicit_application, convert_xor))
  f = parse_expr(raw_expr, transformations=transformations)
  assert list(map(str, f.free_symbols)) == ['x'], \
    'Функция может содержать только одну свободную переменную, x'
  return str(f), autowrap(f)

In [5]:
_, f1 = parse_function("x * cos x")
iterate_upto_precision(left_riemann_sum, f1, 0, 1)

(262144, 0.38177277539957866)